<a href="https://colab.research.google.com/github/ankushKun/learning-ML/blob/master/day3/lin_reg_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear regressing using neural networks

In [1]:
import torch
import torch.nn as nn
import numpy as np

prepare data

In [2]:
inputs = np.array([
                   [56,34,89],[45,89,45],[50,58,56],
                   [122,78,88],[101,67,52],[83,85,36],
                   [42,178,43],[74,74,87],[119,89,90],
                   [95,79,85],[50,110,96],[69,77,89],
                   [57,67,52],[130,90,89],[40,79,100]
],dtype="float32")

targets = np.array([
                   [56,89],[89,45],[50,67],
                   [90,88],[90,52],[96,90],
                   [79,43],[69,37],[112,90],
                   [77,85],[25,65],[88,89],
                   [70,60],[110,67],[95,76]
],dtype="float32")

inputs  = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### TensorDataset
allows access to tensors in form of tuples,and also provides standard APIs to work with datasets

In [3]:
from torch.utils.data import TensorDataset

In [4]:
training = TensorDataset(inputs,targets)
training[0] # input and target at that index

(tensor([56., 34., 89.]), tensor([56., 89.]))

### DataLoader
used to split data into batches if predefined size during training, also provides funtions like shuffle and random sampling

In [5]:
from torch.utils.data import DataLoader

In [6]:
batch_size = 5
train_dl = DataLoader(training,batch_size,shuffle=True)

for i,t in train_dl:
  print(i) #random 5 inputs
  print(t) #random 5 outputs

tensor([[ 74.,  74.,  87.],
        [ 56.,  34.,  89.],
        [ 95.,  79.,  85.],
        [130.,  90.,  89.],
        [ 50., 110.,  96.]])
tensor([[ 69.,  37.],
        [ 56.,  89.],
        [ 77.,  85.],
        [110.,  67.],
        [ 25.,  65.]])
tensor([[ 40.,  79., 100.],
        [ 45.,  89.,  45.],
        [ 69.,  77.,  89.],
        [ 50.,  58.,  56.],
        [101.,  67.,  52.]])
tensor([[95., 76.],
        [89., 45.],
        [88., 89.],
        [50., 67.],
        [90., 52.]])
tensor([[119.,  89.,  90.],
        [ 42., 178.,  43.],
        [ 57.,  67.,  52.],
        [ 83.,  85.,  36.],
        [122.,  78.,  88.]])
tensor([[112.,  90.],
        [ 79.,  43.],
        [ 70.,  60.],
        [ 96.,  90.],
        [ 90.,  88.]])


### nn.Linear


In [7]:
model = nn.Linear(3,2) #(input size,output size)
print(model.weight)
print(model.bias)
print(list(model.parameters()))

Parameter containing:
tensor([[ 0.1979,  0.3782,  0.2449],
        [ 0.0066,  0.0337, -0.2918]], requires_grad=True)
Parameter containing:
tensor([0.0387, 0.3952], requires_grad=True)
[Parameter containing:
tensor([[ 0.1979,  0.3782,  0.2449],
        [ 0.0066,  0.0337, -0.2918]], requires_grad=True), Parameter containing:
tensor([0.0387, 0.3952], requires_grad=True)]


In [8]:
predictions = model(inputs)
print(predictions) # messed up as in previous day

tensor([[ 45.7780, -24.0621],
        [ 53.6283,  -9.4416],
        [ 45.5867, -13.6630],
        [ 75.2394, -21.8526],
        [ 58.1063, -11.8561],
        [ 57.4332,  -6.6993],
        [ 86.2071,  -5.8795],
        [ 63.9804, -22.0121],
        [ 79.2959, -22.0854],
        [ 69.5385, -21.1215],
        [ 75.0500, -23.5841],
        [ 64.6151, -22.5277],
        [ 49.3968, -12.1464],
        [ 81.6066, -21.6874],
        [ 62.3250, -25.8617]], grad_fn=<AddmmBackward>)


### built in mse loss funtion

In [9]:
import torch.nn.functional as F

In [10]:
loss_fn = F.mse_loss

In [11]:
predictions = model(inputs)
loss = loss_fn(predictions,targets)
loss # huge loss XD

tensor(4394.2676, grad_fn=<MseLossBackward>)

### Decreasing loss using optimiser
**torch.optim.SGD**, SGD = Stochastic Gradient Descent, same thing as changing weights and biases by a smol proportional value like day2

In [12]:
opt = torch.optim.SGD(model.parameters(),lr=1e-5) #lr means learning rate

### Training go brrrrr
1. generate predictions
2. calc loss
3. calc gradients
4. adjust weights
5. reset gradients to 0

will work in batches of data

In [13]:
def train(number_epochs,model,loss_fn,opt):
  for epoch in range(number_epochs):
    for i,t in train_dl:
      predictions = model(i)        # generate preds
      loss = loss_fn(predictions,t) # calcs loss
      loss.backward()               # calc grads
      opt.step()                    # update weights and biases, w-=whatever in day2
      opt.zero_grad()               # reset to 0

    if (epoch+1) % 10==0:print(f"{epoch+1}/{number_epochs} | loss: {torch.sqrt(loss)}")

In [14]:
train(100,model,loss_fn,opt)

10/100 | loss: 27.214757919311523
20/100 | loss: 27.020532608032227
30/100 | loss: 28.104780197143555
40/100 | loss: 25.18886375427246
50/100 | loss: 25.468076705932617
60/100 | loss: 19.826688766479492
70/100 | loss: 17.71146011352539
80/100 | loss: 18.887939453125
90/100 | loss: 16.69626235961914
100/100 | loss: 22.824735641479492


In [15]:
predictions = model(inputs)
predictions #better and closer to actual data

tensor([[ 54.6310,  63.0995],
        [ 58.9014,  45.1400],
        [ 53.7551,  48.6319],
        [104.2026,  89.6485],
        [ 83.8753,  65.1340],
        [ 77.1392,  53.6264],
        [ 84.3617,  52.3220],
        [ 76.5121,  72.4263],
        [106.2473,  90.6530],
        [ 89.2598,  79.1946],
        [ 75.8014,  71.9864],
        [ 74.9972,  71.9306],
        [ 59.7597,  50.1355],
        [112.4333,  94.0541],
        [ 61.3780,  67.2091]], grad_fn=<AddmmBackward>)